In [227]:
import numpy as np

In [228]:
db_name = '20k_class_best'
# db_name = '20k_class_no_over'
# db_name = '20k_ner_best'
# db_name = '20k_ner_noover'

In [229]:
all_cats = ['claudience','clpersuasive','clsentiment','clagreement','cldisagreement','clinformative','clmean','clcontroversial', 'cltopic']

In [230]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient[db_name]

In [231]:
for x in mydb['runs'].find({}):
    print(x)

{'_id': 1, 'experiment': {'name': '20k_class_best_claudience', 'base_dir': '/home/group7/masters-thesis/train/ynacc/04_sec_lm/Eval', 'sources': [['eval_model_best_20k.py', ObjectId('5bfec974fd3a196989b5010b')]], 'dependencies': ['dataclasses==0.6', 'fastai==1.0.29.dev0', 'fastprogress==0.1.15', 'matplotlib==3.0.0', 'numpy==1.15.4', 'pandas==0.23.4', 'requests==2.19.1', 'sacred==0.7.4', 'scikit-learn==0.20.0', 'scipy==1.1.0', 'spacy==2.0.16', 'torch==1.0.0.dev20181120'], 'repositories': [], 'mainfile': 'eval_model_best_20k.py'}, 'format': 'MongoObserver-0.7.0', 'command': 'run_exp', 'host': {'hostname': 'idun', 'os': ['Linux', 'Linux-4.15.0-36-generic-x86_64-with-debian-stretch-sid'], 'python_version': '3.6.6', 'cpu': 'Intel(R) Core(TM) i9-7900X CPU @ 3.30GHz', 'gpus': {'gpus': [{'model': 'GeForce GTX 1080 Ti', 'total_memory': 11178, 'persistence_mode': False}, {'model': 'GeForce GTX 1080 Ti', 'total_memory': 11178, 'persistence_mode': False}, {'model': 'GeForce GTX 1080 Ti', 'total_mem

In [232]:
metrics = [('acc', 3), ('f1_macro', 4), ('f1_weighted', 5)]

In [233]:
variances = []
for cat in all_cats:
    best_runs = {}
    aveage_runs = {}
    
    for x in mydb['runs'].find({'experiment.name': db_name + '_' + cat, 'status': 'COMPLETED'}):
        cout_lines = x['captured_out'].split('\n')
        for metric_name, matric_idx in metrics:
            best = 0
            for l in cout_lines:
                l_sp = l.split()
                if len(l_sp) > matric_idx:
                    try:
                        element = l_sp[matric_idx]
                        element = float(element)
                        best = max(best, element)
    #                     print(element)
    #                     print()
                    except ValueError:
                        pass
            if not best is None:
                best_runs[metric_name] = max(best_runs.get(metric_name, 0), best)
                aveage_runs[metric_name] = aveage_runs.get(metric_name, []) + [best]
    
    # best
    print(cat, ' & ' + ' & '.join([str(x) for x in list(best_runs.values())]))
    #     print(aveage_runs)
    # avg of best runs
    print(cat, ' & ' + ' & '.join([str(sum(x) / len(x)) for x in list(aveage_runs.values())]))
    print(cat, ' & ' + ' & '.join([str(np.var(x)) for x in list(aveage_runs.values())]))
    
    [variances.append(np.var(x)) for x in list(aveage_runs.values())]
    
    print()

print()
sum(variances) / len(variances)

claudience  & 0.726334 & 0.637184 & 0.792337
claudience  & 0.7139416000000001 & 0.609091 & 0.7846701999999999
claudience  & 7.038903864000007e-05 & 0.0004251621103999989 & 4.931829095999997e-05

clpersuasive  & 0.857633 & 0.531801 & 0.921369
clpersuasive  & 0.8559178 & 0.518246 & 0.921369
clpersuasive  & 1.1771761599999738e-06 & 7.512650479999975e-05 & 0.0

clsentiment  & 0.619377 & 0.409499 & 0.678197
clsentiment  & 0.614879 & 0.40141340000000003 & 0.6735692
clsentiment  & 1.3887055999999825e-05 & 4.2550173039999936e-05 & 8.134439760000047e-06

clagreement  & 0.915952 & 0.724504 & 0.938069
clagreement  & 0.9108061428571429 & 0.7077948571428571 & 0.937701142857143
clagreement  & 5.043570122448988e-06 & 0.00012094010840816345 & 8.119132653061187e-07

cldisagreement  & 0.730703 & 0.714719 & 0.736081
cldisagreement  & 0.7262435999999999 & 0.710855 & 0.7311021999999999
cldisagreement  & 8.943931040000015e-06 & 1.570651320000002e-05 & 2.660610856000018e-05

clinformative  & 0.852487 & 0.646

9.369621242799684e-05